https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data

https://www.kaggle.com/shahules/basic-eda-cleaning-and-glove

In [ ]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense


In [ ]:
!unzip /content/train.tsv.zip

Archive:  /content/train.tsv.zip
  inflating: train.tsv               


In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('train.tsv',sep='\t')

In [ ]:
df[:10]

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [ ]:
df.drop_duplicates(subset='SentenceId',keep='first',inplace=True)


In [ ]:
df

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1
...,...,...,...,...
155984,155985,8540,... either you 're willing to go with this cla...,2
155997,155998,8541,"Despite these annoyances , the capable Claybur...",2
156021,156022,8542,-LRB- Tries -RRB- to parody a genre that 's al...,1
156031,156032,8543,The movie 's downfall is to substitute plot fo...,1


In [ ]:
df.Phrase

0         A series of escapades demonstrating the adage ...
63        This quiet , introspective and entertaining in...
81        Even fans of Ismail Merchant 's work , I suspe...
116       A positively thrilling combination of ethnogra...
156       Aggressive self-glorification and a manipulati...
                                ...                        
155984    ... either you 're willing to go with this cla...
155997    Despite these annoyances , the capable Claybur...
156021    -LRB- Tries -RRB- to parody a genre that 's al...
156031    The movie 's downfall is to substitute plot fo...
156039    The film is darkly atmospheric , with Herrmann...
Name: Phrase, Length: 8529, dtype: object

In [ ]:
import re

In [ ]:
phraselist=[]

for phrase in df.Phrase:
  phrase=phrase.lower()
  phrase=re.sub(r'[^\w]', ' ', phrase)
  phraselist.append(phrase)

len(phraselist)

  

8529

In [ ]:
df['Phraselist']=phraselist

In [ ]:
df

,PhraseId,SentenceId,Phrase,Sentiment,Phraselist
0,1,1,A series of escapades demonstrating the adage ...,1,a series of escapades demonstrating the adage ...
63,64,2,"This quiet , introspective and entertaining in...",4,this quiet introspective and entertaining in...
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1,even fans of ismail merchant s work i suspe...
116,117,4,A positively thrilling combination of ethnogra...,3,a positively thrilling combination of ethnogra...
156,157,5,Aggressive self-glorification and a manipulati...,1,aggressive self glorification and a manipulati...
...,...,...,...,...,...
155984,155985,8540,... either you 're willing to go with this cla...,2,either you re willing to go with this cla...
155997,155998,8541,"Despite these annoyances , the capable Claybur...",2,despite these annoyances the capable claybur...
156021,156022,8542,-LRB- Tries -RRB- to parody a genre that 's al...,1,lrb tries rrb to parody a genre that s al...
156031,156032,8543,The movie 's downfall is to substitute plot fo...,1,the movie s downfall is to substitute plot fo...


In [ ]:
output=len(df.Sentiment.unique())
output

5

In [ ]:
y=df.Sentiment.values
y.shape

(8529,)

In [ ]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

In [ ]:
df['Phraselist']=df['Phraselist'].apply(lambda x : remove_URL(x))

In [ ]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)


In [ ]:
df['Phraselist']=df['Phraselist'].apply(lambda x : remove_html(x))

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
df['Phraselist']=df['Phraselist'].apply(lambda x: remove_emoji(x))

In [ ]:
!pip install pyspellchecker

     |████████████████████████████████| 2.5MB 5.8MB/s 


In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

In [ ]:
df['Phraselist']=df['Phraselist'].apply(lambda x : correct_spellings(x))

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
df


,PhraseId,SentenceId,Phrase,Sentiment,Phraselist
0,1,1,A series of escapades demonstrating the adage ...,1,a series of escapades demonstrating the adage ...
1,64,2,"This quiet , introspective and entertaining in...",4,this quiet introspective and entertaining inde...
2,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1,even fans of ismail merchant i work i suspect ...
3,117,4,A positively thrilling combination of ethnogra...,3,a positively thrilling combination of ethnogra...
4,157,5,Aggressive self-glorification and a manipulati...,1,aggressive self glorification and a manipulati...
...,...,...,...,...,...
8524,155985,8540,... either you 're willing to go with this cla...,2,either you re willing to go with this claustro...
8525,155998,8541,"Despite these annoyances , the capable Claybur...",2,despite these annoyances the capable clayburgh...
8526,156022,8542,-LRB- Tries -RRB- to parody a genre that 's al...,1,lab tries rob to parody a genre that i already...
8527,156032,8543,The movie 's downfall is to substitute plot fo...,1,the movie i downfall is to substitute plot for...


In [ ]:
!wget 'https://nlp.stanford.edu/data/wordvecs/glove.twitter.27B.zip'

--2021-01-21 04:16:56--  https://nlp.stanford.edu/data/wordvecs/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.twitter.27B.zip [following]
--2021-01-21 04:16:57--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408741 (1.4G) [application/zip]
Saving to: ‘glove.twitter.27B.zip’

glove.twitter.27B.z 100%[===================>]   1.42G  2.11MB/s    in 11m 45s 

2021-01-21 04:28:42 (2.06 MB/s) - ‘glove.twitter.27B.zip’ saved [1520408741/1520408741]



In [ ]:
!unzip '/content/glove.twitter.27B.zip'

Archive:  /content/glove.twitter.27B.zip
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  


In [ ]:
import numpy as np


In [ ]:
embeddings_index = dict()
embeddings_lookup= dict()
f = open('glove.twitter.27B.200d.txt')
for i,line in enumerate(f):
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
	embeddings_lookup[word] = i

f.close()

In [ ]:

df['Phraselist'].apply(lambda x : len(x.split())).values.max()

48

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def CleanTokenize(df):
    head_lines = list()
    lines = df["Phraselist"].values.tolist()
    for line in lines:
        #line = clean_text(line)
        # tokenize the text
        tokens = word_tokenize(line)
        # remove puntuations
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove non alphabetic characters
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        # remove stop words
        words = [w for w in words if not w in stop_words]
        head_lines.append(words)
    return head_lines
head_lines = CleanTokenize(df)
head_lines[0:10]

[['series',
  'escapades',
  'demonstrating',
  'adage',
  'good',
  'goose',
  'also',
  'good',
  'gander',
  'occasionally',
  'amuses',
  'none',
  'amounts',
  'much',
  'story'],
 ['quiet', 'introspective', 'entertaining', 'independent', 'worth', 'seeking'],
 ['even',
  'fans',
  'ismail',
  'merchant',
  'work',
  'suspect',
  'would',
  'hard',
  'time',
  'sitting',
  'one'],
 ['positively',
  'thrilling',
  'combination',
  'ethnography',
  'intrigue',
  'betrayal',
  'deceit',
  'murder',
  'shakespearean',
  'tragedy',
  'juicy',
  'soap',
  'opera'],
 ['aggressive', 'self', 'glorification', 'manipulative', 'whitewash'],
 ['comedy',
  'drama',
  'nearly',
  'epic',
  'proportions',
  'rooted',
  'sincere',
  'performance',
  'title',
  'character',
  'undergoing',
  'midlife',
  'crisis'],
 ['narrative', 'trouble', 'every', 'day', 'plodding', 'mess'],
 ['importance',
  'earnest',
  'thick',
  'wit',
  'plays',
  'like',
  'reading',
  'bartlett',
  'familiar',
  'quotations

In [ ]:
np.shape(head_lines)


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(8529,)

In [ ]:
embeddings_lookup

{'<user>': 0,
 '.': 1,
 ':': 2,
 'rt': 3,
 ',': 4,
 '<repeat>': 5,
 '<hashtag>': 6,
 '<number>': 7,
 '<url>': 8,
 '!': 9,
 'i': 10,
 'a': 11,
 '"': 12,
 'the': 13,
 '?': 14,
 'you': 15,
 'to': 16,
 '(': 17,
 '<allcaps>': 18,
 '<elong>': 19,
 ')': 20,
 'me': 21,
 'de': 22,
 '<smile>': 23,
 '！': 24,
 'que': 25,
 'and': 26,
 '。': 27,
 '-': 28,
 'my': 29,
 'no': 30,
 '、': 31,
 'is': 32,
 'it': 33,
 '…': 34,
 'in': 35,
 'n': 36,
 'for': 37,
 '/': 38,
 'of': 39,
 'la': 40,
 "'s": 41,
 '*': 42,
 'do': 43,
 "n't": 44,
 'that': 45,
 'on': 46,
 'y': 47,
 "'": 48,
 'e': 49,
 'o': 50,
 'u': 51,
 'en': 52,
 'this': 53,
 'el': 54,
 'so': 55,
 'be': 56,
 "'m": 57,
 'with': 58,
 'just': 59,
 '>': 60,
 'your': 61,
 '^': 62,
 'like': 63,
 'have': 64,
 'te': 65,
 'at': 66,
 '？': 67,
 'love': 68,
 'se': 69,
 'are': 70,
 '<': 71,
 'm': 72,
 'r': 73,
 'if': 74,
 'all': 75,
 'b': 76,
 '・': 77,
 'not': 78,
 'but': 79,
 'we': 80,
 'es': 81,
 'ya': 82,
 '&': 83,
 'follow': 84,
 'up': 85,
 'what': 86,
 'get': 87

In [ ]:
x=np.zeros((8529,48))
i=0
for idx,words in enumerate(head_lines):

  for idy,w in enumerate(words):
    if(w in embeddings_lookup.keys()):
      x[idx][idy]=embeddings_lookup[w]
    else:i=i+1



In [ ]:
i

1346

In [ ]:
len(line)

200

In [ ]:
vocab_size=len(embeddings_lookup)
embedding_matrix = np.zeros((vocab_size, 200))
l=0
for i,line in enumerate(embeddings_index.values()):
  if len(line)<200:
    l=l+1
    continue
  embedding_matrix[i] = line
print(embedding_matrix.shape)



(1193515, 200)


In [ ]:
l

1

In [ ]:
vocab_size

1193515

In [ ]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 200, input_length=48,weights=[embedding_matrix],trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(output, activation='sigmoid'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 48, 200)           238703000 
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1228928   
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 325       
Total params: 239,940,509
Trainable params: 1,237,509
Non-trainable params: 238,703,000
_________________________________________________________________
None


In [ ]:
model.compile(optimizer='adam', loss='SparseCategoricalCrossentropy', metrics=['acc'])
model.fit(x, y, epochs=100, verbose=1)

Epoch 1/100
267/267 [==============================] - 4s 2ms/step - loss: 1.7172 - acc: 0.2631
Epoch 2/100
267/267 [==============================] - 1s 2ms/step - loss: 1.4398 - acc: 0.3548
Epoch 3/100
267/267 [==============================] - 1s 2ms/step - loss: 1.3340 - acc: 0.4178
Epoch 4/100
267/267 [==============================] - 1s 2ms/step - loss: 1.2711 - acc: 0.4591
Epoch 5/100
267/267 [==============================] - 1s 2ms/step - loss: 1.2067 - acc: 0.4970
Epoch 6/100
267/267 [==============================] - 1s 2ms/step - loss: 1.1521 - acc: 0.5248
Epoch 7/100
267/267 [==============================] - 1s 3ms/step - loss: 1.0951 - acc: 0.5512
Epoch 8/100
267/267 [==============================] - 1s 2ms/step - loss: 1.0236 - acc: 0.5871
Epoch 9/100
267/267 [==============================] - 1s 2ms/step - loss: 0.9993 - acc: 0.5939
Epoch 10/100
267/267 [==============================] - 1s 2ms/step - loss: 0.9370 - acc: 0.6223
Epoch 11/100
267/267 [=================

In [ ]:
review='it just feels so over the top, hammy and unearned. it doesnt feel lovingly hammy it feels accidentally cheesy.'
review=review.split()
new=np.zeros((1,48))
for idy,w in enumerate(review):
    if(w in embeddings_lookup.keys()):
      new[0][idy]=embeddings_lookup[w]
  

In [ ]:
y_predict=model.predict(new)
np.argmax(y_predict)

1